In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/cataract_model.keras')

In [13]:
    %%writefile app.py

    import streamlit as st
    import pandas as pd
    import numpy as np
    import pickle
    import sklearn

    # Load the trained model
    # Ensure 'LinearRegressionModel.pkl' is in the same directory as this app.py file
    try:
        with open('/content/LinearRegressionModel.pkl', 'rb') as file:
            pipe = pickle.load(file)
    except FileNotFoundError:
        st.error("Model file 'LinearRegressionModel.pkl' not found. Please make sure it's in the same directory.")
        st.stop()
    except Exception as e:
        st.error(f"Error loading the model: {e}")
        st.stop()

    # Load the cleaned car data to get unique values for the dropdowns
    try:
        car = pd.read_csv('/content/Cleaned_Car_data.csv')
    except FileNotFoundError:
        st.error("Data file 'Cleaned_Car_data.csv' not found. Please make sure it's in the same directory.")
        st.stop()
    except Exception as e:
        st.error(f"Error loading the data: {e}")
        st.stop()

    st.set_page_config(
        page_title="Car Price Predictor",
        page_icon="🚗",
        layout="centered",
        initial_sidebar_state="expanded"
    )

    # Custom CSS for a better look and feel
    st.markdown("""
        <style>
        .main-header {
            font-size: 2.5rem;
            font-weight: bold;
            color: #1a237e;
            text-align: center;
            margin-bottom: 0.5em;
        }
        .stSelectbox, .stNumberInput {
            background-color: #e8eaf6;
            border-radius: 10px;
            padding: 10px;
            box-shadow: 2px 2px 5px rgba(0,0,0,0.2);
        }
        .stButton>button {
            background-color: #4a148c;
            color: white;
            font-weight: bold;
            border-radius: 10px;
            padding: 12px 20px;
            transition: all 0.3s ease-in-out;
        }
        .stButton>button:hover {
            background-color: #6a1b9a;
            box-shadow: 0 4px 8px rgba(0,0,0,0.3);
        }
        .prediction-box {
            background-color: #c5cae9;
            padding: 20px;
            border-radius: 10px;
            text-align: center;
            font-size: 1.5rem;
            font-weight: bold;
            color: #1a237e;
            margin-top: 2em;
            box-shadow: 4px 4px 10px rgba(0,0,0,0.3);
        }
        </style>
    """, unsafe_allow_html=True)

    st.markdown('<p class="main-header">Car Price Predictor</p>', unsafe_allow_html=True)
    st.write("Enter the details of the car to get a predicted price.")

    # Get unique values for dropdowns from the dataframe
    companies = sorted(car['company'].unique())
    car_names = sorted(car['name'].unique())
    fuel_types = car['fuel_type'].unique()
    years = sorted(car['year'].unique(), reverse=True)

    # Create the input widgets for the app
    company = st.selectbox('Select Company', companies)

    # Filter car names based on the selected company
    filtered_cars = sorted(car[car['company'] == company]['name'].unique())
    name = st.selectbox('Select Car Name', filtered_cars)

    year = st.number_input('Year of Purchase', min_value=1995, max_value=2025, value=2010, step=1, format="%d")
    kms_driven = st.number_input('Kilometers Driven', min_value=0, value=50000, step=1000)
    fuel_type = st.radio('Fuel Type', fuel_types)

    # Create a button to trigger the prediction
    if st.button('Predict Price'):
        # Create a DataFrame with the user's input
        input_data = pd.DataFrame(
            [[name, company, year, kms_driven, fuel_type]],
            columns=['name', 'company', 'year', 'kms_driven', 'fuel_type']
        )

        # Make the prediction
        try:
            prediction = pipe.predict(input_data)[0]
            # Display the result
            if prediction < 0:
                st.markdown(f'<div class="prediction-box">Prediction not possible for these values.</div>', unsafe_allow_html=True)
            else:
                st.markdown(f'<div class="prediction-box">The predicted price is: ₹{np.round(prediction, 2):,.2f}</div>', unsafe_allow_html=True)
        except Exception as e:
            st.error(f"Prediction failed: {e}")



Overwriting app.py


In [8]:
!pip install pyngrok

In [17]:
from pyngrok import ngrok

# Set authentication token if you haven't already done so
ngrok.set_auth_token("32NbhrsP5OTYNJ70dEKbKvnWCYn_2KS3aMvvSXjCWatkDvcdm")

# Start Streamlit server on a specific port
!nohup streamlit run app.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://9e0773eea632.ngrok-free.app
